Problem 1: [7 marks]
1. Use California Housing Dataset:
from sklearn.datasets import fetch_california_housing
2. Perform an 80:20 train/test split, followed by an 80:20 train/validation split from the
trainset. Perform standard Scaling on all splits.
3. Perform Linear Regression and Non-Linear Regression using an appropriate Polynomial
curve to reduce error on the Train and Validation Datasets. To prevent Overfitting, use L2
Regularizer with appropriate Lambda values. (Optimize for 100 Epochs)
4. Plot the loss vs epoch curve on different parameters of Polynomial degree and Lambda
values on the Train and validation sets to choose appropriate values.
5. Predict on the Test set for any top-3 fine-tuned models.
6. Print SSE, R2 scores for Train, Validation, and Test Sets for top-3 Models